In [51]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [52]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [53]:
# Python Master for ISPStore
sheet = client.open_by_key("1T_fuAUdJw-DB9dn9v3ZRg5YZ3mruCtg3Ghkd1mhVBRM") # Master Sheets ( Dimensions Sheet)

In [54]:
Schema_Master = sheet.worksheet("Schema_Master")
Schema_Master = get_as_dataframe(Schema_Master)

In [55]:
Schema_Master= Schema_Master[Schema_Master['flag']==1]
Schema_Master.head()

Project                 SQL Schema Name  From Date    To Date  \
0              Cholayil            [CholayilManagement]  21-Jul-25  20-Aug-25   
1              PIDILITE            [PidiliteManagement]   1-Oct-25  21-Oct-25   
2             Danone-GT            [DanoneGTManagement]  21-Jul-25  20-Aug-25   
3  Himalaya Turmeric GT  [HimalayaTurmericGTManagement]  21-Jul-25  20-Aug-25   
4  Himalaya Turmeric MT    [HimalayaTurmericManagement]  26-Jul-25  25-Aug-25   

   flag Unnamed: 5 Unnamed: 6 Unnamed: 7   28 au  
0   1.0     =D2-C2        NaN        NaN  15 Oct  
1   1.0     =D3-C3  16-Jul-25  15-Aug-25     NaN  
2   1.0     =D4-C4        NaN        NaN     NaN  
3   1.0     =D5-C5        NaN        NaN     NaN  
4   1.0     =D6-C6        NaN        NaN     NaN

In [56]:
x= Schema_Master['SQL Schema Name'].to_list()
projects = [str(p).strip("[]") for p in Schema_Master['Project'].tolist()]


In [57]:
date_ranges = [
    (
        pd.to_datetime(row["From Date"]).strftime("%Y-%m-%d"),
        pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")
    )
    for _, row in Schema_Master.iterrows()
]


In [58]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [59]:
conn = pyodbc.connect(conn_str)

In [60]:
# x = ['[CholayilManagement]','[ParleManagement]']

In [61]:
final_df = pd.DataFrame()

In [62]:
# date_ranges = [
#     ("2025-07-21", "2025-07-22"),
#      ("2025-08-22","2025-08-24")
# ]

In [63]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f""" 
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        A.FullName,
        M.Gender,
        A.Region,
        A.State,
        A.City,
        A.Designation,
        M.LastWorkingDate,
        M.Status AS EmpStatus,
        A.Status AS AttRawStatus,
        A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
            WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
            WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
            WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
            WHEN A.Status = 'Vacant' THEN 'V'
            WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
            WHEN A.Status = 'Casual Leave' THEN 'CL'
            WHEN A.Status = 'Half Day' THEN 'HD'
            ELSE 'Condition not found'
        END AS AttStatus,
        DATEDIFF(
            DAY, 
            CAST('{date_from}' AS DATE), 
            TRY_CONVERT(date, A.[Date], 105)
        ) + 1 AS DayNum, 
        TRY_CONVERT(date, A.[Date], 105) AS Date2,
        RANK() OVER (PARTITION BY A.UserName,A.[Date] ORDER BY A.ISPAttendanceMasterExportId DESC) AS rk
    FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
    LEFT JOIN PowerBI_LISV2.{value}.UserMaster AS M
        ON A.UserName = M.UserName
    WHERE 
        TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
        AND A.UserName NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager')
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%'
)
SELECT * FROM CTE WHERE rk = 1;"""
# TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
# AND A.UserName LIKE '%SUP%'
    # A.UserName LIKE '%ISP%' AND 
# AND M.Status = 'True'
# MONTH(TRY_CONVERT(date, A.[Date], 105)) = 08   
# FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd') AS DayNum,
        df = pd.read_sql(query, conn)
        final_df = pd.concat([final_df, df], ignore_index=True)
        print(f"Running for: {value} {date_from} {date_to}")

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_16308\2431663319.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: [CholayilManagement] 2025-07-21 2025-08-20
Running for: [PidiliteManagement] 2025-10-01 2025-10-21
Running for: [DanoneGTManagement] 2025-07-21 2025-08-20
Running for: [HimalayaTurmericGTManagement] 2025-07-21 2025-08-20
Running for: [HimalayaTurmericManagement] 2025-07-26 2025-08-25
Running for: [ReckittManagement] 2025-07-16 2025-08-15
Running for: [PiramalManagement] 2025-07-26 2025-08-25
Running for: [WiproGranammaManagement] 2025-07-26 2025-08-25
Running for: [DanoneMTManagement] 2025-07-16 2025-08-15
Running for: [AdaniWilmerManagement] 2025-07-16 2025-08-15
Running for: [AmulManagement] 2025-07-21 2025-08-20
Running for: [ZydusMTManagement] 2025-07-16 2025-08-15
Running for: [ZydusMTManagement] 2025-07-16 2025-08-15
Running for: [HaleonManagement] 2025-07-26 2025-08-25
Running for: [BeamSuntoryManagement] 2025-08-01 2025-08-31
Running for: [ZydusGTManagement] 2025-07-21 2025-08-20
Running for: [IDAMManagement] 2025-07-26 2025-08-25
Running for: [EvereadyManagement] 

In [64]:
# final_df2=final_df.drop_duplicates(['UserName','Date'])
final_df.tail()

SchemaName        UserName    EmployeeID       FullName Gender Region  \
282066   Dabur GT  DBR-ISPStar010  YM2025009799  Chandan Kumar      M  North   
282067   Dabur GT  DBR-ISPStar010  YM2025009799  Chandan Kumar      M  North   
282068   Dabur GT  DBR-ISPStar010  YM2025009799  Chandan Kumar      M  North   
282069   Dabur GT  DBR-ISPStar010  YM2025009799  Chandan Kumar      M  North   
282070   Dabur GT  DBR-ISPStar010  YM2025009799  Chandan Kumar      M  North   

                State   City Designation      LastWorkingDate EmpStatus  \
282066  Uttar Pradesh  NOIDA    Promoter  18-06-2025 00:00:00     False   
282067  Uttar Pradesh  NOIDA    Promoter  18-06-2025 00:00:00     False   
282068  Uttar Pradesh  NOIDA    Promoter  18-06-2025 00:00:00     False   
282069  Uttar Pradesh  NOIDA    Promoter  18-06-2025 00:00:00     False   
282070  Uttar Pradesh  NOIDA    Promoter  18-06-2025 00:00:00     False   

       AttRawStatus ApprovalStatus AttStatus  DayNum       Date2  rk  
282066       Absent        Pending         A      26  2025-06-26   1  
282067       Absent        Pending         A      27  2025-06-27   1  
282068       Absent        Pending         A      28  2025-06-28   1  
282069       Absent        Pending         A      29  2025-06-29   1  
282070       Absent        Pending         A      30  2025-06-30   1

In [65]:
# check = final_df.groupby(['UserName', 'Date']).size().reset_index(name='Count')
# SAMPLE CHEK ON COUNT

In [66]:
# check2 = check[(check['Count'] == 2) & (check['UserName'].str.contains('CH'))]

In [67]:
# check2.head(10)

In [68]:
final_df.info() #256725
# 232885 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282071 entries, 0 to 282070
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   SchemaName       282071 non-null  object
 1   UserName         282071 non-null  object
 2   EmployeeID       271718 non-null  object
 3   FullName         282071 non-null  object
 4   Gender           282043 non-null  object
 5   Region           282071 non-null  object
 6   State            282071 non-null  object
 7   City             282071 non-null  object
 8   Designation      282040 non-null  object
 9   LastWorkingDate  40648 non-null   object
 10  EmpStatus        282043 non-null  object
 11  AttRawStatus     275014 non-null  object
 12  ApprovalStatus   282071 non-null  object
 13  AttStatus        282071 non-null  object
 14  DayNum           282071 non-null  int64 
 15  Date2            282071 non-null  object
 16  rk               282071 non-null  int64 
dtypes: int64(2

In [69]:
final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendance.csv')

In [70]:
# SAMPLE CHEK
# QUERY = """
# SELECT TOP (10) * FROM PowerBI_LISV2.[CholayilManagement].ISPAttendanceMasterExport where UserName= 'CH-ISP-195' and
# FORMAT(TRY_CONVERT(date, [Date], 105), 'dd-MM-yyyy')= '06-08-2025';
# """
# dfX = pd.read_sql(QUERY, conn)

In [71]:
# dfX.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\dfX.csv')

In [72]:
# query = f""" 
# WITH CTE AS (
#     SELECT
#         '{project}' AS SchemaName,
#         A.UserName,
#         M.EmployeeID,
#         A.FullName,
#         M.Gender,
#         A.Region,
#         A.State,
#         A.City,
#         A.Designation,
#         M.LastWorkingDate,
#         M.Status AS EmpStatus,
#         A.Status AS AttRawStatus,
#         A.ApprovalStatus,
#         CASE
#             WHEN A.Status = 'Present' THEN 'P'
#             WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
#             WHEN A.Status = 'Leave' THEN 'L'
#             WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
#             WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
#             WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
#             WHEN A.Status = 'Training' THEN 'T'
#             WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
#             WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
#             WHEN A.Status = 'Vacant' THEN 'V'
#             WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
#             WHEN A.Status = 'Casual Leave' THEN 'CL'
#             WHEN A.Status = 'Half Day' THEN 'HD'
#             ELSE 'Condition not found'
#         END AS AttStatus,
#         TRY_CONVERT(date, A.[Date], 105) AS Date2,
#         RANK() OVER (PARTITION BY A.UserName ORDER BY TRY_CONVERT(date, A.[Date], 105) DESC, A.ISPAttendanceMasterExportId DESC) AS rk
#     FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
#     LEFT JOIN PowerBI_LISV2.{value}.UserMaster AS M
#         ON A.UserName = M.UserName
#     WHERE 
#         TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
#         AND A.UserName NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager')
#         AND LOWER(A.UserName) NOT LIKE '%test%'
#         AND LOWER(A.UserName) NOT LIKE '%sup%'
#         AND LOWER(A.FullName) NOT LIKE '%test%'
# )
# SELECT * FROM CTE WHERE rk = 1;

In [73]:
# DATEDIFF(
#             DAY, 
#             CAST('{date_from}' AS DATE), 
#             TRY_CONVERT(date, A.[Date], 105)
#         ) + 1 AS DayNum,   

In [74]:
final_df = final_df.fillna("")
index_cols = final_df.columns[0:11].tolist()
# Clean up
df["DayNum"] = pd.to_numeric(df["DayNum"], errors="coerce")
# df = df.dropna(subset=["DayNum"])
df["DayNum"] = df["DayNum"].astype(int)
df["AttStatus"] = df["AttStatus"].str.strip().str.upper()

In [76]:
#sparsh pivot
pivot_map = {}
for _, row in final_df.iterrows():
    key = tuple(row[col] for col in index_cols)
    day = row.get("DayNum", "")
    status = row.get("AttStatus", "")
    if key not in pivot_map:
        pivot_map[key] = {}
    if day in pivot_map[key]:
        # merge multiple statuses safely
        existing = pivot_map[key][day]
        combined = ",".join(sorted(set(filter(None, (existing + "," + status).split(",")))))
        pivot_map[key][day] = combined
    else:
        pivot_map[key][day] = status

In [77]:
# Build DataFrame from the mapping (this creates only existing day columns, sparse construction)
pivot_df = pd.DataFrame.from_dict(pivot_map, orient="index")

In [78]:
index_df = pd.DataFrame(list(pivot_map.keys()), columns=index_cols)
result = pd.concat([index_df.reset_index(drop=True), pivot_df.reset_index(drop=True)], axis=1)

In [79]:
result.head()

SchemaName    UserName    EmployeeID                       FullName Gender  \
0   Cholayil   CH-ISP-08  YM2024000025  Bhagyashree Narayan Kshirsgar      F   
1   Cholayil   CH-ISP-09  YM2024000008           Suvarna Sonu Shelake      F   
2   Cholayil  CH-ISP-102  YM2024000018  Rajendr Chandrakant Galedvaru      M   
3   Cholayil  CH-ISP-103  YM2024000055         Kalpana Gajanan Vaidya      F   
4   Cholayil  CH-ISP-106  YM2024000125                         Aastha      F   

  Region          State       City       Designation LastWorkingDate  ...  3  \
0   West    Maharashtra     Nashik          Promoter                  ...  P   
1   West    Maharashtra     Mumbai  Sharing Promoter                  ...  P   
2   West    Maharashtra    Solapur          Promoter                  ...  P   
3   West    Maharashtra       Pune          Promoter                  ...  P   
4  North  Uttar Pradesh  Ghaziabad          Promoter                  ...  L   

    4  5  6  7   8  9  10  11   32  
0  WO  P  P  P   P  P   P  WO  NaN  
1   P  P  P  P   P  P  WO   L  NaN  
2   P  P  P  P  WO  P   P   P  NaN  
3  WO  P  P  P   P  P   P  WO  NaN  
4   P  P  L  P  WO  P   P   P  NaN  

[5 rows x 43 columns]

In [ ]:
# # Create the pivot table
# final_df = final_df.fillna("")
# pivot_df = pd.pivot_table(
#     final_df,
#     index=df.columns[0:11].tolist(),  # Columns B to L
#     columns="DayNum",
#     values="AttStatus",
#     aggfunc=lambda x: ", ".join(x.astype(str).unique()),
#     dropna=False  # Combine unique status values
# ).reset_index()

In [80]:
result=result.fillna("")

In [81]:
# Sort columns (optional) so DayNum columns are numeric-sorted if they are numeric strings
# day_cols = [c for c in result.columns if c not in index_cols]
# try:
#     sorted_day_cols = sorted(day_cols, key=lambda x: int(x) if x!="" else -1)
# except Exception:
#     sorted_day_cols = sorted(day_cols)

# result = pd.concat([result[index_cols], result[sorted_day_cols]], axis=1)

# Sort columns properly
day_cols = sorted([c for c in pivot_df.columns if isinstance(c, (int, float))])
result = result[index_cols + day_cols]

In [82]:
# result.head()

In [83]:
result.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendancepivot_df.csv')